<a href="https://colab.research.google.com/github/angelmorales2621-gif/Investigaci-n-de-operaciones-/blob/main/almac%C3%A9n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import sympy as sp
from scipy.optimize import minimize, fsolve

class MultiItemInventoryOptimizer:
    def __init__(self, K, D, h, a, A):
        """
        Parámetros:
        K: costos de preparación
        D: tasas de demanda
        h: costos de almacenamiento
        a: áreas por unidad
        A: área total disponible
        """
        self.K = np.array(K)
        self.D = np.array(D)
        self.h = np.array(h)
        self.a = np.array(a)
        self.A = A
        self.n = len(K)

    def no_restringidos(self):
        """Paso 1: Calcular valores óptimos no restringidos"""
        y_star = np.sqrt(2 * self.K * self.D / self.h)
        return y_star

    def  restricciones(self, y_values):
        """Paso 2: Verificar si satisface la restricción"""
        total_area = np.sum(self.a * y_values)
        return total_area <= self.A, total_area

    def lagrange(self, vars):
        """
        Función de Lagrange para el problema

        vars = [y1, y2, ..., yn, lambda]
        """
        n = self.n
        y = vars[:n]
        lam = vars[n]

        # Función objetivo: costo total
        TCU = np.sum(self.K * self.D / y + self.h * y / 2)

        # Restricción
        restriccion = np.sum(self.a * y) - self.A

        # Función de Lagrange
        L = TCU - lam * restriccion

        return L

    def ecuaciones_gradiente(self, vars):
        """
        Ecuaciones del gradiente para resolver con multiplicadores de Lagrange
        """
        n = self.n
        y = vars[:n]
        lam = vars[n]

        ecuaciones = []

        # ∂L/∂y_i = 0
        for i in range(n):
            eq = -self.K[i] * self.D[i] / (y[i]**2) + self.h[i]/2 - lam * self.a[i]
            ecuaciones.append(eq)

        # ∂L/∂λ = 0 (restricción)
        ec_restr = np.sum(self.a * y) - self.A
        ecuaciones.append(ec_restr)

        return ecuaciones

    def multiplicadores(self, inicial=None):
        """
        Paso 3: Resolver con multiplicadores de Lagrange
        """
        if inicial is None:
            # Usar solución no restringida
            y_inicial = self.no_restringidos()
            lam_inicial = -0.1  # λ debe ser negativo
            inicial = np.append(y_inicial, lam_inicial)

        # Resolver sistema de ecuaciones
        solucion = fsolve(self.ecuaciones_gradiente, inicial)

        y_optimo = solucion[:self.n]
        lambda_optimo = solucion[self.n]

        return y_optimo, lambda_optimo

    def minimizar(self):
        """
        Resolver mediante minimización directa con restricción
        """
        # Función objetivo
        def objetivo(y):
            return np.sum(self.K * self.D / y + self.h * y / 2)

        # Restricción
        restric = {
            'type': 'ineq',
            'fun': lambda y: self.A - np.sum(self.a * y)
        }

        # Límites (y_i > 0)
        bounds = [(1e-6, None) for _ in range(self.n)]

        # Guess inicial (solución no restringida)
        x0 = self.no_restringidos()

        # Minimizar
        result = minimize(objetivo, x0, method='SLSQP',
                         bounds=bounds, constraints=restric)

        return result.x

    def solve(self):
        """Resolver el problema completo"""

        # Paso 1: Solución no restringida
        print("\n1. SOLUCIÓN NO RESTRINGIDA:")
        y_norestringida = self.no_restringidos()
        for i in range(self.n):
            print(f"   Artículo {i+1}: y_{i+1}* = {y_norestringida[i]:.4f}")

        # Paso 2: Verificar restricción
        print("\n2. VERIFICACIÓN DE RESTRICCIÓN:")
        satisfies, area_total = self.restricciones(y_norestringida)
        print(f"   Área requerida: ∑a_i·y_i = {area_total:.4f}")
        print(f"   Área disponible: A = {self.A}")
        print(f"   ¿Satisface restricción? {satisfies}")

        if satisfies:
            print("\n✓ La solución no restringida es óptima")
            return y_norestringida, 0

        # Paso 3: Solución restringida
        print("\n3. SOLUCIÓN RESTRINGIDA (Multiplicadores de Lagrange):")

        # Método 1: Sistema de ecuaciones
        print("\n   Método 1: Resolviendo sistema de ecuaciones del gradiente...")
        y_restringida, lambda_opt = self.multiplicadores()

        print(f"   Multiplicador de Lagrange (λ): {lambda_opt:.6f}")
        print("\n   Cantidades óptimas restringidas:")
        for i in range(self.n):
            print(f"   Artículo {i+1}: y_{i+1}* = {y_restringida[i]:.4f}")

        # Verificar restricción
        area_restr = np.sum(self.a * y_restringida)
        print(f"\n   Área utilizada: ∑a_i·y_i = {area_restr:.4f}")
        print(f"   Diferencia con área disponible: {area_restr - self.A:.6f}")

        # Método 2: Minimización directa
        print("\n   Método 2: Minimización directa con restricción...")
        y_min = self.minimizar()
        print("\n   Cantidades óptimas (minimización directa):")
        for i in range(self.n):
            print(f"   Artículo {i+1}: y_{i+1}* = {y_min[i]:.4f}")

        # Calcular costos
        print("\n4. COMPARACIÓN DE COSTOS:")
        costo_sinrestr = np.sum(self.K * self.D / y_norestringida + self.h * y_norestringida / 2)
        costo_conrestr = np.sum(self.K * self.D / y_restringida + self.h * y_restringida / 2)
        costo_min = np.sum(self.K * self.D / y_min + self.h * y_min / 2)

        print(f"   Costo no restringido: ${costo_sinrestr:.4f}")
        print(f"   Costo restringido (Lagrange): ${costo_conrestr:.4f}")
        print(f"   Costo restringido (minimización): ${costo_min:.4f}")

        return y_restringida, lambda_opt


# Ejemplo 11.2-3
def ejemplo_11_2_3():

    # Datos del ejemplo
    K = [10, 5, 15]          # Costos de preparación ($)
    D = [2, 4, 4]           # Tasas de demanda (unidades/día)
    h = [0.30, 0.10, 0.20]  # Costos de almacenamiento ($/unidad/día)
    a = [1, 1, 1]           # Área por unidad (pies²)
    A = 25                  # Área total disponible (pies²)

    # Crear optimizador
    optimizar = MultiItemInventoryOptimizer(K, D, h, a, A)

    # Resolver
    return optimizar.solve()


# Versión alternativa con fórmula explícita
def solucion(K, D, h, a, A):
    """
    Resuelve usando la fórmula explícita derivada de las condiciones de primer orden
    """

    print("SOLUCIÓN CON FÓRMULA EXPLÍCITA")


    K = np.array(K)
    D = np.array(D)
    h = np.array(h)
    a = np.array(a)

    # Función para calcular y_i dado λ
    def y_i(lam):
        return np.sqrt(2 * K * D / (h - 2 * lam * a))

    # Función para encontrar λ que satisface la restricción
    def funcion_restr(lam):
        y = y_i(lam)
        return np.sum(a * y) - A

    # Buscar λ usando bisección (λ debe ser negativo y tal que h - 2λa > 0)
    # Primero encontrar límites para λ
    lam_max = min(h / (2 * a)) * 0.99
    lam_min = -10

    # Verificar signos en los extremos
    f_min = funcion_restr(lam_min)
    f_max = funcion_restr(lam_max)

    print(f"   f(λ_min={lam_min}) = {f_min:.6f}")
    print(f"   f(λ_max={lam_max}) = {f_max:.6f}")

    # Usar bisección para encontrar λ
    lam_l, lam_a = lam_min, lam_max

    for _ in range(50):  # 50 iteraciones máximas
        lam_mid = (lam_l + lam_a) / 2
        f_mid = funcion_restr(lam_mid)

        if abs(f_mid) < 1e-8:
            break

        if f_mid > 0:
            lam_l = lam_mid
        else:
            lam_a = lam_mid

    lambda_opt = lam_mid
    y_opt = y_i(lambda_opt)

    print(f"\n   Multiplicador de Lagrange óptimo: λ = {lambda_opt:.6f}")
    print("\n   Cantidades óptimas:")
    for i in range(len(K)):
        print(f"   Artículo {i+1}: y_{i+1}* = {y_opt[i]:.4f}")

    area_total = np.sum(a * y_opt)
    print(f"\n   Área utilizada: ∑a_i·y_i = {area_total:.4f}")
    print(f"   Diferencia con área disponible: {area_total - A:.6f}")

    return y_opt, lambda_opt


if __name__ == "__main__":
    # Ejecutar el ejemplo
    y_opt, lambda_opt = ejemplo_11_2_3()

    # También resolver con fórmula explícita
    K = [10, 5, 15]
    D = [2, 4, 4]
    h = [0.30, 0.10, 0.20]
    a = [1, 1, 1]
    A = 25

    solucion(K, D, h, a, A)


    print("""

    1. Cálculo de valores óptimos no restringidos:
       y_i* = √(2K_iD_i/h_i)

    2. Verificación de restricción:
       Si ∑a_i·y_i* ≤ A, la solución es óptima

    3. Si no se satisface, usar multiplicadores de Lagrange:
       a) Formular función de Lagrange
       b) Resolver sistema de ecuaciones ∂L/∂y_i = 0 y ∂L/∂λ = 0
       c) La solución da: y_i* = √[2K_iD_i/(h_i - 2λ*a_i)]

    """)


1. SOLUCIÓN NO RESTRINGIDA:
   Artículo 1: y_1* = 11.5470
   Artículo 2: y_2* = 20.0000
   Artículo 3: y_3* = 24.4949

2. VERIFICACIÓN DE RESTRICCIÓN:
   Área requerida: ∑a_i·y_i = 56.0419
   Área disponible: A = 25
   ¿Satisface restricción? False

3. SOLUCIÓN RESTRINGIDA (Multiplicadores de Lagrange):

   Método 1: Resolviendo sistema de ecuaciones del gradiente...
   Multiplicador de Lagrange (λ): 0.100000

   Cantidades óptimas restringidas:
   Artículo 1: y_1* = -467156.0444
   Artículo 2: y_2* = 93840.3859
   Artículo 3: y_3* = 373340.6585

   Área utilizada: ∑a_i·y_i = 25.0000
   Diferencia con área disponible: 0.000000

   Método 2: Minimización directa con restricción...

   Cantidades óptimas (minimización directa):
   Artículo 1: y_1* = 6.3397
   Artículo 2: y_2* = 7.0878
   Artículo 3: y_3* = 11.5725

4. COMPARACIÓN DE COSTOS:
   Costo no restringido: $10.3631
   Costo restringido (Lagrange): $-28047.3212
   Costo restringido (minimización): $13.6238
SOLUCIÓN CON FÓRMULA E

/tmp/ipython-input-1692506173.py:85: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  solucion = fsolve(self.ecuaciones_gradiente, inicial)
